# PBAT Framework Demonstration

This notebook demonstrates the key components and capabilities of the Profile-Based Adaptive Testing (PBAT) framework.

## Overview
PBAT is an AI-driven quiz generation platform that combines:
- Retrieval-Augmented Generation (RAG) for factual accuracy
- Item Response Theory (IRT) for psychometric calibration
- Multi-Armed Restless Bandit (MARB) for adaptive personalization

In [ ]:
# Import required libraries
import sys
sys.path.append('..')

from pbat_main import *
from visualization import PBATVisualizer
from analysis import PBATAnalyzer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("PBAT Framework loaded successfully!")

## 1. Document Processing Module Demo

In [ ]:
# Initialize document processor
doc_processor = DocumentProcessor()

# Sample educational content
sample_docs = [
    "Machine learning algorithms learn patterns from data to make predictions. Supervised learning uses labeled training data, while unsupervised learning finds hidden patterns in unlabeled data.",
    "Neural networks consist of interconnected nodes that process information. Deep learning uses multiple layers to extract hierarchical features from complex data.",
    "Cloud computing provides on-demand access to computing resources over the internet. Virtualization enables efficient resource sharing and scalability."
]

# Process documents
chunks = doc_processor.parse_documents(sample_docs)
embeddings = doc_processor.extract_embeddings(chunks)

print(f"Processed {len(chunks)} text chunks")
print(f"Generated {embeddings.shape[0]} embeddings with {embeddings.shape[1]} features")

# Test retrieval
query = "neural networks"
relevant_chunks = doc_processor.retrieve_relevant_chunks(query, top_k=3)
print(f"\nTop relevant chunks for '{query}':")
for i, chunk in enumerate(relevant_chunks):
    print(f"{i+1}. {chunk['text'][:100]}...")

## 2. Quiz Generation Module Demo

In [ ]:
# Initialize quiz generator
quiz_generator = QuizGenerator(doc_processor)

# Generate questions for different topics and difficulties
topics = ["Machine Learning", "Neural Networks", "Cloud Computing"]
difficulties = ["Easy", "Medium", "Hard"]

for topic in topics:
    print(f"\n=== {topic} Questions ===")
    for difficulty in difficulties:
        questions = quiz_generator.generate_questions(topic, difficulty, count=2)
        validated_questions = quiz_generator.validate_questions(questions)
        
        print(f"\n{difficulty} Level:")
        for i, q in enumerate(validated_questions):
            print(f"{i+1}. {q.text}")
            print(f"   IRT Params: a={q.discrimination:.2f}, b={q.difficulty:.2f}, c={q.guessing:.2f}")

## 3. PBAT Adaptive Module Demo

In [ ]:
# Initialize PBAT adapter
pbat_adapter = PBATAdapter(quiz_generator)

# Create a sample learner profile
learner_id = "demo_student"
profile = pbat_adapter.initialize_learner_profile(learner_id, initial_ability=0.0)

print(f"Initialized learner profile: {profile}")

# Generate a question bank
all_questions = []
for topic in ["Machine Learning", "Neural Networks"]:
    for difficulty in ["Easy", "Medium", "Hard"]:
        questions = quiz_generator.generate_questions(topic, difficulty, count=5)
        all_questions.extend(quiz_generator.validate_questions(questions))

print(f"\nGenerated question bank with {len(all_questions)} questions")

# Simulate adaptive session
session_questions = []
simulated_responses = []

print("\n=== Adaptive Session Simulation ===")

for step in range(8):  # Simulate 8 questions
    # Select next question
    available = [q for q in all_questions if q not in session_questions]
    next_question = pbat_adapter.select_next_question(learner_id, available)
    
    if next_question is None:
        break
    
    # Simulate learner response (based on IRT model)
    current_ability = pbat_adapter.learner_profiles[learner_id]['ability']
    prob_correct = pbat_adapter.irt_probability(current_ability, next_question)
    response = np.random.random() < prob_correct
    
    # Update learner model
    pbat_adapter.update_ability_map(learner_id, response, next_question)
    
    session_questions.append(next_question)
    simulated_responses.append(response)
    
    updated_ability = pbat_adapter.learner_profiles[learner_id]['ability']
    
    print(f"Step {step+1}:")
    print(f"  Question: {next_question.text[:60]}...")
    print(f"  Difficulty: {next_question.difficulty:.2f}")
    print(f"  Response: {'Correct' if response else 'Incorrect'}")
    print(f"  Updated Ability: {current_ability:.3f} → {updated_ability:.3f}")
    print()

# Check stopping rule
should_stop = pbat_adapter.check_stopping_rule(learner_id)
print(f"Should stop assessment: {should_stop}")
print(f"Final ability estimate: {pbat_adapter.learner_profiles[learner_id]['ability']:.3f}")

## 4. Full Experiment Simulation

In [ ]:
# Run a small-scale experiment
simulator = ExperimentSimulator()
results_df = simulator.run_experiment(n_learners=20, n_attempts=3)

print(f"Generated {len(results_df)} experimental records")
print("\nFirst 10 records:")
print(results_df.head(10))

## 5. Performance Analysis

In [ ]:
# Statistical analysis
analyzer = PBATAnalyzer(results_df)
summary = analyzer.quick_summary()

print("=== PBAT Performance Summary ===")
for metric, value in summary['pbat_performance'].items():
    print(f"{metric}: {value:.2f}")

print("\n=== Model Comparison ===")
comparison = results_df.groupby('Model').agg({
    'AvgScore': ['mean', 'std'],
    'CognitiveLoad': ['mean', 'std'],
    'HallucinationRate': ['mean', 'std'],
    'Retention48h': ['mean', 'std'],
    'FeedbackScore': ['mean', 'std']
}).round(2)

print(comparison)

## 6. Visualization

In [ ]:
# Create visualizations
viz = PBATVisualizer(results_df)

# Model comparison plot
plt.figure(figsize=(15, 10))

metrics = ['AvgScore', 'CognitiveLoad', 'HallucinationRate', 'Retention48h', 'FeedbackScore']

for i, metric in enumerate(metrics):
    plt.subplot(2, 3, i+1)
    sns.boxplot(data=results_df, x='Model', y=metric)
    plt.title(f'{metric} by Model')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Learning trajectory
plt.figure(figsize=(12, 8))

trajectory_metrics = ['AvgScore', 'Retention48h']
for i, metric in enumerate(trajectory_metrics):
    plt.subplot(1, 2, i+1)
    
    trajectory_data = results_df.groupby(['Model', 'AttemptNo'])[metric].mean().reset_index()
    
    for model in results_df['Model'].unique():
        model_data = trajectory_data[trajectory_data['Model'] == model]
        plt.plot(model_data['AttemptNo'], model_data[metric], 
                marker='o', linewidth=2, label=model)
    
    plt.title(f'{metric} Learning Trajectory')
    plt.xlabel('Attempt Number')
    plt.ylabel(metric)
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Key Findings

Based on the simulation, we can observe:

1. **Adaptive Question Selection**: PBAT dynamically adjusts question difficulty based on learner ability
2. **Reduced Hallucination**: RAG-based validation ensures factual accuracy
3. **Improved Learning Outcomes**: Higher retention and satisfaction scores
4. **Efficient Assessment**: Adaptive stopping rules optimize quiz length

## Next Steps

- Run larger-scale experiments with more learners
- Test with real educational content and student data
- Fine-tune hyperparameters for optimal performance
- Implement additional evaluation metrics

In [ ]:
# Save results for further analysis
results_df.to_csv('demo_experiment_results.csv', index=False)
print("Demo results saved to 'demo_experiment_results.csv'")
print("\nDemo completed successfully!")